# Beta version
Blur only the face (and not a rectangle across the face)

In [ ]:
# import the libraries
import face_recognition
import cv2
import numpy as np
import os
from os.path import isfile, join
import re
from PIL import Image, ImageDraw

In [ ]:
# Open the input video file
input_video = cv2.VideoCapture('./data/trump3.mp4')

In [ ]:
# Store all frames as images
n = 0

while(input_video.isOpened()):
    ret, frame = input_video.read()
    
    if ret == False:
        break
    
    cv2.imwrite('./frames/frame' + str(n) + '.jpg', frame)
    n += 1

In [ ]:
# Load images
pathIn = './frames/'

frame_array = []
files = [f for f in os.listdir(pathIn)]

files.sort(key = lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

files = files[1:]

for i in range(len(files)):
    filename = pathIn + files[i]
    
    # Reading each files
    img = cv2.imread(filename)
    
    # Inserting the frames into an image array
    frame_array.append(img)

In [ ]:
i = 0

#Loop over all images to create mask
for image in frame_array:
    face_locations = face_recognition.face_locations(image)
    face_landmarks_list = face_recognition.face_landmarks(image)
    
    for face_location in face_locations:

        # Print the location of each face in this image
        top, right, bottom, left = face_location
        
        # Create empty mask
        mask = np.zeros(image.shape, dtype=np.uint8)
        
       # Create a PIL imagedraw object so we can draw on the picture
        pil_image = Image.fromarray(mask)
        d = ImageDraw.Draw(pil_image)
        
    for face_landmarks in face_landmarks_list:

        left_eyebrow_top = min(face_landmarks['left_eyebrow'], key = lambda t: t[1])
        right_eyebrow_top = min(face_landmarks['right_eyebrow'], key = lambda t: t[1])

        points = []
        points.insert(0, left_eyebrow_top)
        points.insert(0, right_eyebrow_top)
        points.insert(0, face_landmarks['chin'])

        cleaned = [j for i in points for j in i]

        d.polygon(cleaned, fill = 'white')
    
    # saving mask
    i+=1 
    pil_image = pil_image.save('./masks/mask' + str(i) + '.jpg')

In [ ]:
# Load masks
path = './masks/'

mask_array = []
masks = [f for f in os.listdir(path)]

masks.sort(key = lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

masks = masks[1:]
for i in range(len(masks)):
    filename = path + masks[i]
    
    # Reading each files
    img = cv2.imread(filename)
    
    # Inserting the frames into an image array
    mask_array.append(img)

In [ ]:
m = 0

for img, mask in zip(frame_array, mask_array):
    # Apply mask
    blur = cv2.GaussianBlur(img, (45,45), 30)
    blurred_img = np.where(mask != 0, blur, img)
  
    m += 1
    
    cv2.imwrite('./frames_masked/mask' + str(m) + '.jpg', blurred_img)

In [ ]:
# Load images with blurred faces
path = './frames_masked/'

frames_masked = []
masked = [f for f in os.listdir(path)]

masked.sort(key = lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

for i in range(len(masked)):
    filename = path + masked[i]
    
    #reading each files
    img = cv2.imread(filename)
    
    #inserting the frames into an image array
    frames_masked.append(img)
    


In [ ]:
fps = input_video.get(cv2.CAP_PROP_FPS)

width  = int(input_video.get(3))
height = int(input_video.get(4))

out = cv2.VideoWriter('./result/blurred_trump.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))

for i in range(len(frames_masked)):
    out.write(frames_masked[i])

input_video.release()
out.release()